(00-backprop)=
# Backpropagation

In this notebook, we look at the **backpropagation algorithm** for efficient gradient computation on computational graphs. Backpropagation involves local message passing of outputs in the forward pass, and gradients in the backward pass. The resulting time complexity is linear in the number of size of the network, i.e. the total number of weights and neurons for neural networks. Neural networks are computational graphs with nodes for differentiable operations. This fact allows scaling training large neural networks. We will implement a minimal scalar-valued **autograd engine** and a neural net library on top it to train a small regression model.

## BP on computational graphs

A neural network can be modelled as a **directed acyclic graph** (DAG) of nodes that implements a function $f$, i.e. all computation flows from an input $\boldsymbol{\mathsf{x}}$ to an output node $f(\boldsymbol{\mathsf{x}})$ with no cycles. 
During training, this is extended to implement the calculation of the loss.
Recall that our goal is to obtain parameter node values $\hat{\boldsymbol{\Theta}}$ after optimization (e.g. with SGD) such that the $f_{\hat{\boldsymbol{\Theta}}}$ minimizes the expected value of a loss function $\ell.$ Backpropagation allows us to efficiently compute $\nabla_{\boldsymbol{\Theta}} \ell$ for SGD after $(\boldsymbol{\mathsf{x}}, y) \in \mathcal{B}$ is passed to the input nodes.

```{figure} ../../../img/nn/03-comp-graph.png
---
width: 80%
name: compute
align: center
---
Computational graph of a dense layer. Note that parameter nodes (yellow) always have zero fan-in.
```

**Forward pass.** Forward pass computes $f_{\boldsymbol{\Theta}}(\boldsymbol{\mathsf{x}}).$ All compute nodes are executed starting from the input nodes (which evaluates to the input vector $\boldsymbol{\mathsf x}$). This passed to its child nodes, and so on up to the loss node. The output value of each node is stored to avoid recomputation for child nodes that depend on the same node. This also preserves the network state for backward pass. Finally, forward pass builds the computational graph which is stored in memory. It follows that forward pass for one input is roughly $O(E)$ in time and memory where $E$ is the number of edges of the graph.

**Backward pass.** Backward computes gradients starting from the loss node $\ell$ down 
to the input nodes $\boldsymbol{\mathsf{x}}.$ 
The gradient of $\ell$ with respect to itself is $1$. This serves as the base step.
For any other node $u$ in the graph, we can assume that the **global gradient**
${\partial \ell}/{\partial v}$ is cached for each node $v \in N_u$, where $N_u$ are all nodes 
in the graph that depend on $u$. On the other hand, the **local gradient**
${\partial v}/{\partial u}$ between adjacent nodes is specified 
analytically based on the functional
dependence of $v$ upon $u.$ These are computed at runtime given current node values
cached during forward pass.

The global gradient with respect to node $u$ can then be inductively calculated using the chain 
rule:

$$
\frac{\partial \ell}{\partial u} = \sum_{v \in N_u} \frac{\partial \ell}{\partial v} \frac{\partial v}{\partial u}.
$$

This can be visualized as gradients flowing from the loss node to each network node. 
The flow of gradients will end on parameter and input nodes which depend on no other
nodes. These are called **leaf nodes**. It follows that the algorithm terminates.

```{figure} ../../../img/backward-1.svg
---
width: 80%
name: backward-1
align: center
---
Computing the global gradient for a single node. Note that gradient type is distinguished by color: **local** (red) and **global** (blue).
```

This can be visualized as gradients flowing to each network node from the loss node. The flow of gradients will end on parameter and input nodes which have zero fan-in. Global gradients are stored in each compute node in the `grad` attribute for use by the next layer, along with node values obtained during forward pass which are used in local gradient computation. Memory can be released after the weights are updated. On the other hand, there is no need to store local gradients as these are computed as needed. Backward pass can be implemented roughly as follows:

```python
class Node:
    ...

    def sorted_graph(self):
        """Return toposorted comp graph with self as root."""
        ...

    def backward(self):
        self.grad = 1.0
        for node in self.sorted_graph():
            for parent in node._parents:
                parent.grad += node.grad * node._local_grad(parent)

```

Each node has to wait for all incoming gradients from dependent nodes before passing the gradient to its parents. This is done by topologically sorting the nodes based on dependency with `self` as root (i.e. the node calling `backward` is always treated as the terminal node). 
The contributions of each child node are then accumulated based on the chain rule, where
`node.grad` is the global gradient which is equal to `∂self / ∂node`, while the local gradient `node._local_grad(parent)` is equal to `∂node / ∂parent`. By construction, each child node occurs before any of its parent nodes, thus the full gradient of a child node is calculated before it is sent to its parent nodes ({numref}`03-parent-child-nodes`).

```{figure} ../../../img/nn/03-parent-child-nodes.png
---
width: 100%
name: 03-parent-child-nodes
---
Equivalent ways of computing the global gradient. On the left, the global gradient is computed by tracking the dependencies from $u$ to each of its child node during forward pass. This is our formal statement before. Algorithmically, we start from each node in the upper layer, and we contribute one term in the sum to each parent node. Eventually, all terms in the chain rule is accumulated and the parent node fires, sending gradients to its parent nodes in the previous layer.
```

To construct the topologically sorted list of nodes from a terminal node, we use [depth-first search](https://www.geeksforgeeks.org/topological-sorting/). The following example is shown in {numref}`00-toposort`:

In [3]:
parents = {
    "a": [],
    "b": [],
    "x": [],
    "c": ["a", "b"],
    "d": ["x", "c"],
    "e": ["c"],
    "f": ["d"]
}

def sorted_graph(self):
    """Return toposorted comp graph with self as root."""
    topo = []
    visited = list()
    def dfs(node):
        if node not in visited:
            visited.append(node)
            print("v", visited)
            for parent in parents[node]:
                dfs(parent)
            topo.append(node)
            print("t", topo)
    dfs(self)
    return reversed(topo)

list(sorted_graph("f"))

v ['f']
v ['f', 'd']
v ['f', 'd', 'x']
t ['x']
v ['f', 'd', 'x', 'c']
v ['f', 'd', 'x', 'c', 'a']
t ['x', 'a']
v ['f', 'd', 'x', 'c', 'a', 'b']
t ['x', 'a', 'b']
t ['x', 'a', 'b', 'c']
t ['x', 'a', 'b', 'c', 'd']
t ['x', 'a', 'b', 'c', 'd', 'f']


['f', 'd', 'c', 'b', 'a', 'x']

```{figure} ../../../img/nn/00-toposort.png
---
width: 100%
name: 00-toposort
---
Graph encoded in the `parents` dictionary above. Note `d` which `f` has no dependence on is excluded.
Visited nodes (shaded in red) starts from the terminal node backwards into the graph. Then, the nodes are pushed forwards once all leaf nodes (no parents, shaded yellow) are visited.
```

<br>

## Properties of backpropagation

Some characteristics of backprop which explains why it is ubiquitous in deep learning:

* **Modularity.** Backprop is a useful tool for reasoning about gradient flow and can suggest ways to improve training or network design. Moreover, since it only requires local gradients between nodes, it allows modularity when designing deep neural networks. 
In other words, we can (in principle) arbitrarily connect layers of computation.

* **Runtime.** Each edge is the DAG is passed exactly once ({numref}`03-parent-child-nodes`). Hence, the time complexity for finding global gradients is $O(n_\mathsf{E})$ where $n_\mathsf{E}$ is the number of edges in the graph, where we assume that each compute node and local gradient evaluation is constant time. For fully-connected networks, $n_\mathsf{E} = n_\mathsf{M} + n_\mathsf{V}$ where $n_\mathsf{M}$ is the number of weights and $n_\mathsf{V}$ is the number of activations. It follows that one backward pass for an instance is proportional to the network size.

* **Memory.** Each training step naively requires $O(2 n_\mathsf{E})$ memory since we store both gradients and values. This can be improved by releasing the gradients and activations of non-leaf nodes in the previous layer once a layer finishes computing its gradient. 

* **GPU parallelism.** Note that forward computation can generally be parallelized in the batch dimension and often times in the layer dimension. This can leverage massive parallelism in the GPU significantly decreasing runtime by trading off memory. The same is true for backward pass which can also be expressed in terms of matrix multiplications! {eq}`backprop-output`

<br>

## References

{cite}`timviera` {cite}`backprop-offconvex` {cite}`pytorch-autograd` {cite}`micrograd`